In [ ]:
# !pip install opencv-python
# !pip install tensorflow


In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import torch
from torchvision import datasets, transforms
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

#training and testing data paths

In [ ]:
train_data_path= '/content/drive/My Drive/data2/training_data/'
test_data_path= '/content/drive/My Drive/testing_data/'

class_names = sorted(os.listdir(train_data_path))
n_classes=len(class_names)



In [ ]:
print(class_names)
print(n_classes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
36


#Making training dataframe

In [ ]:

train_image_paths = []
train_labels = []

for folder_name in os.listdir(train_data_path):
    folder_path = os.path.join(train_data_path, folder_name)
    if os.path.isdir(folder_path):
        images = os.listdir(folder_path)
        for image_name in images:
            image_path = os.path.join(folder_path, image_name)
            train_image_paths.append(image_path)
            train_labels.append(class_names.index(folder_name))


train_data = {'image_path': train_image_paths, 'label': train_labels}
train_df = pd.DataFrame(train_data)

#Making test dataframe

In [ ]:
test_paths = []
test_y = []

for folder_name in os.listdir(test_data_path):
    folder_path = os.path.join(test_data_path, folder_name)
    if os.path.isdir(folder_path):
        images = os.listdir(folder_path)
        for image_name in images:
            image_path = os.path.join(folder_path, image_name)
            test_paths.append(image_path)
            test_y.append(class_names.index(folder_name))

test_data = {'image_path': test_paths, 'label': test_y}
test_df = pd.DataFrame(test_data)



#Making validation data by splitting test data

In [ ]:

test_images = test_df['image_path']
test_l = test_df['label']

test_image_paths, valid_image_paths, test_labels, valid_labels = train_test_split(
    test_images, test_l, test_size=0.2, random_state=42)

test_data = {'image_path': test_image_paths, 'label': test_labels}
test_df = pd.DataFrame(test_data)

valid_data = {'image_path': valid_image_paths, 'label': valid_labels}
valid_df = pd.DataFrame(valid_data)

In [ ]:
test_image_paths=test_image_paths.tolist()
test_labels=test_labels.tolist()

valid_image_paths=valid_image_paths.tolist()
valid_labels=valid_labels.tolist()

#Function to customize dataset

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        label = self.labels[index]
        return image, label

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 16
learning_rate = 0.001
num_epochs = 15

#transform function
transform = transforms.Compose([
    transforms.Resize((120, 120)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


#making dataloaders

In [ ]:

train_dataset = CustomDataset(train_image_paths, train_labels, transform=transform)
test_dataset=CustomDataset(test_image_paths, test_labels, transform=transform)
valid_dataset=CustomDataset(valid_image_paths, valid_labels, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True)

Visualizing data

In [ ]:
print(len(train_loader))
print(len(test_loader))
print(len(valid_loader))

646
51
13


In [ ]:
for batch in train_loader:
  # print(batch)
  x,y=batch
  print(x.shape)
  print(y)
  break

torch.Size([32, 3, 120, 120])
tensor([34, 30,  0,  9,  7,  9, 26, 15, 20, 20,  3, 15,  7, 15,  3, 12,  6, 15,
         1, 25, 33, 10, 18,  0, 19,  9, 32, 23,  4, 29, 14,  3])


In [ ]:
for batch in valid_loader:
  # print(batch)
  x,y=batch
  print(x.shape)
  print(y)
  break

torch.Size([16, 3, 120, 120])
tensor([ 1,  7, 24, 18, 19, 19, 25, 23, 19, 13, 35, 29, 17,  0, 33, 26])


In [ ]:
for batch in test_loader:
  # print(batch)
  x,y=batch
  print(x.shape)
  print(y)
  break

torch.Size([16, 3, 120, 120])
tensor([ 4, 34,  3, 19, 15,  0, 21, 14,  8, 17, 18, 23, 27, 12, 16, 14])


#Building an ANN model from pytorch

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3 * 120 * 120, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 36)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [ ]:

model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

#Training our ANN model and Validating also with every epoch

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    # Training
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # if (i + 1) % 2 == 0:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}")

    train_accuracy = 100 * correct / total
    average_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = 100 * val_correct / val_total
    average_val_loss = val_loss / len(valid_loader)

    # Print training and validation statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {average_loss:.4f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {average_val_loss:.4f}, "
          f"Validation Accuracy: {val_accuracy:.2f}%")

Epoch [1/15], Training Loss: 2.5896, Training Accuracy: 55.03%, Validation Loss: 1.2871, Validation Accuracy: 86.63%
Epoch [2/15], Training Loss: 0.9914, Training Accuracy: 87.12%, Validation Loss: 0.4788, Validation Accuracy: 94.06%
Epoch [3/15], Training Loss: 0.5721, Training Accuracy: 90.23%, Validation Loss: 0.3018, Validation Accuracy: 94.06%
Epoch [4/15], Training Loss: 0.4409, Training Accuracy: 91.35%, Validation Loss: 0.2336, Validation Accuracy: 94.55%
Epoch [5/15], Training Loss: 0.3756, Training Accuracy: 92.23%, Validation Loss: 0.1774, Validation Accuracy: 97.52%
Epoch [6/15], Training Loss: 0.3348, Training Accuracy: 92.71%, Validation Loss: 0.1541, Validation Accuracy: 97.52%
Epoch [7/15], Training Loss: 0.3066, Training Accuracy: 93.14%, Validation Loss: 0.1335, Validation Accuracy: 97.52%
Epoch [8/15], Training Loss: 0.2859, Training Accuracy: 93.44%, Validation Loss: 0.1179, Validation Accuracy: 98.02%
Epoch [9/15], Training Loss: 0.2696, Training Accuracy: 93.74%, 

#Testing the ANN model

In [ ]:
# Test the model
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

# Calculate test accuracy
test_accuracy = 100 * test_correct / test_total
average_test_loss = test_loss / len(test_loader)

# Print test statistics
print(f"Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.0828, Test Accuracy: 98.14%


#Saving the model

In [ ]:
save_dir = 'ANN_model_for_characters'
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(save_dir, 'model')
torch.save(model, save_path)

#Function to predict image
(takes the path of picture and model)

In [ ]:

def predict_image_label(image_path, model):
    transform = transforms.Compose([
        transforms.Resize((120, 120)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    model.eval()

    with torch.no_grad():
        output = model(image)

        _, predicted = torch.max(output, 1)
        return class_names[predicted.item()]


In [ ]:
print(predict_image_label('/content/drive/My Drive/data2/testing_data/0/37908.png',model))


0
